In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import ast
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import numpy as np
import xarray as xr
from datetime import datetime

# tk libs
from lib.objs.tkpaths import Site
from lib.objs.predictor import Predictor
from lib.io.matlab import ReadGowMat
from lib.waves import Calculate_TWL
from lib.KMA import Persistences
from lib.extremes import FitGEV_KMA_Frechet as FitGEV
from lib.extremes import SampleGEV_KMA_Smooth as SampleGEV
from lib.extremes import ChromosomesProbabilities_KMA as ChromProbs



# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_est_pred = ST.ESTELA.pred_slp        # estela slp predictor

p_wvs_parts = ST.WAVES.partitions_p1  # original wave partitions (hs, tp)
p_wvs_parts_noTCs = ST.WAVES.partitions_notcs # wave partitions (TCs removed)
p_wvs_fams_noTCs = ST.WAVES.families_notcs    # wave families (TCs removed)




#p_wvs_parts = ST.WAVES.partitions_p1
#p_hist_r2_params = ST.TCs.hist_r2_params  # hist storms parameters inside r2

# output files
#p_wvs_parts_noTCs = ST.WAVES.partitions_notcs
#p_wvs_fams_noTCs = ST.WAVES.families_notcs

# wave families sectors
#wvs_sectors = ast.literal_eval(PR.WAVES.sectors)

# date limits for TCs removal from waves data, and TC time window (hours)
#tc_rm_date1 = PR.WAVES.tc_remov_date1
#tc_rm_date2 = PR.WAVES.tc_remov_date2
#tc_time_window = int(PR.WAVES.tc_remov_timew)

In [2]:
# --------------------------------------
# Load data

# load ESTELA predictor KMA 
pred = Predictor(p_est_pred)
pred.Load()
xds_KMA = pred.KMA

# load original wave partitions (hourly data)
xds_wvs_pts = ReadGowMat(p_wvs_parts)

# load wave families (sea, swl1, swl2) without TCs
xds_wvs_fams_noTCs = xr.open_dataset(p_wvs_fams_noTCs)



# TODO: QUITAR
# --------------------------------------
# Load test data

# TESTEAR CON DATOS MATLAB
from lib.io.matlab import ReadMatfile
from lib.custom_dateutils import DateConverter_Mat2Py as dmp
from lib.custom_dateutils import datevec2datetime as d2d

# load test KMA (bmus, time, number of centers)
p_test_mat = '/Users/nico/Projects/TESLA-kit/source/teslakit/data/tests/test_ExtremesGEV/Nico_Montecarlo/'
dmatf = ReadMatfile(op.join(p_test_mat, 'bmus_testearpython.mat'))
xds_KMA = xr.Dataset(
    {
        'bmus':('time', dmatf['KMA']['bmus']),
    },
    coords = {'time':np.array(d2d(dmatf['KMA']['Dates']))}
)
n_clusters = len(dmatf['KMA']['order'])

# load test wave families (sea, swl1, swl2 (Hs, Tp, Dir), time)
dmatf = ReadMatfile(op.join(p_test_mat, 'KWA_waves_2PART_TCs_nan.mat'))
xds_wvs_fam_noTCs = xr.Dataset(
    {
        'sea_Hs':('time', dmatf['sea']['Hs']),
        'sea_Tp':('time', dmatf['sea']['Tp']),
        'sea_Dir':('time', dmatf['sea']['Dir']),
        'swell_1_Hs':('time', dmatf['swl1']['Hs']),
        'swell_1_Tp':('time', dmatf['swl1']['Tp']),
        'swell_1_Dir':('time', dmatf['swl1']['Dir']),
        'swell_2_Hs':('time', dmatf['swl2']['Hs']),
        'swell_2_Tp':('time', dmatf['swl2']['Tp']),
        'swell_1_Dir':('time', dmatf['swl2']['Dir']),
        
    },
    coords = {'time': dmp(dmatf['time'])},
)


# TODO: aqui perdemos datos TWL maximo (al pasar los datos de 3h a 1d), y estamos con extremos....
# Select data at KMA time
xds_wvs_fam_noTCs = xds_wvs_fam_noTCs.sel(time=xds_KMA.time)
xds_wvs_pts = xds_wvs_pts.sel(time=xds_KMA.time)


# TODO: DATOS TEST MATLAB, BUSCAR/ORDENAR LOS REALES
# load Simulation data: dates simulation, bmus simulation, MJO (categ, rmm1, rmm2), AWT (PCs123, bmus)
dmatf = ReadMatfile(op.join(p_test_mat, 'DWT_1000years_mjo_awt_v2.mat'))
xds_DWT_MJO_AWT = xr.Dataset(
    {
        'daily_WT':(('n_sims','time',), dmatf['bmusim']),
        
    },
    coords = {'time': dmp(dmatf['datesim'].astype(float))},
)


In [3]:
# --------------------------------------
# Calculate Max TWL for each storm

# TODO: def function

# calculate total water level from waves data
xda_TWL = Calculate_TWL(xds_wvs_pts.hs, xds_wvs_pts.tp)

# locate dates where KMA WT changes (bmus series)
bmus_diff = np.diff(xds_KMA.bmus.values)
ix_ch = np.where((bmus_diff != 0))[0]+1
ix_ch = np.insert(ix_ch, 0,0)  
ds_ch = xds_KMA.time.values[ix_ch]  # dates where WT changes

# list of tuples with (date start, date end) for each WT window
dates_tup_WT = [(ds_ch[c], ds_ch[c+1]-np.timedelta64(1,'D')) for c in range(len(ds_ch)-1)]
dates_tup_WT.append((dates_tup_WT[-1][1]+np.timedelta64(1,'D'), xds_KMA.time.values[-1]))


# find max TWL inside each WT window and store wave families data
TWL_WT_max = []
times_WT_max = []
for d1, d2 in dates_tup_WT:
    
    # get TWL inside WT window
    wt_TWL = xda_TWL.sel(time=slice(d1,d2))[:] 
    
    # get window maximum TWL date
    wt_max_TWL = wt_TWL.where(wt_TWL==wt_TWL.max(), drop=True).squeeze()
    max_TWL = wt_max_TWL.values
    max_date = wt_max_TWL.time.values
        
    # append data
    TWL_WT_max.append(max_TWL)
    times_WT_max.append(max_date)
    
# select waves families data at WT max TWL instants
xds_WT_wvs_fam_noTCs = xds_wvs_fam_noTCs.sel(time=times_WT_max)
xds_WT_wvs_fam_noTCs['max_TWL'] = ('time', TWL_WT_max)
print(xds_WT_wvs_fam_noTCs)

# select kma at WT max TWL instants
xds_WT_KMA = xds_KMA.sel(time=times_WT_max)
print(xds_WT_KMA)


<xarray.Dataset>
Dimensions:      (time: 6527)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-22 1979-01-23 ... 2016-12-27
Data variables:
    sea_Hs       (time) float64 0.07675 0.2218 nan nan ... nan 0.4055 0.0714
    sea_Tp       (time) float64 3.593 5.279 nan nan ... 3.554 nan 8.607 3.947
    sea_Dir      (time) float64 154.1 152.3 nan nan ... 132.7 nan 130.9 155.6
    swell_1_Hs   (time) float64 0.7545 1.045 0.7774 ... 0.5823 0.9433 0.788
    swell_1_Tp   (time) float64 12.39 12.8 13.57 12.36 ... 12.79 11.48 11.53
    swell_1_Dir  (time) float64 65.3 68.52 67.25 68.16 ... 60.52 70.03 64.44
    swell_2_Hs   (time) float64 2.825 3.079 2.524 1.812 ... 3.336 2.696 2.331
    swell_2_Tp   (time) float64 9.993 9.377 9.595 9.11 ... 8.238 8.36 9.482 9.51
    max_TWL      (time) float64 0.7318 0.8098 0.7329 ... 0.7256 0.7005 0.6512
<xarray.Dataset>
Dimensions:  (time: 6527)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-22 1979-01-23 ... 2016-12-27
Data variables:
    

In [4]:
# --------------------------------------
# Fit each variable to GEV and sample GEV parameters

vars_fit = ['sea_Hs', 'sea_Tp', 'swell_1_Hs', 'swell_1_Tp', 'swell_2_Hs', 'swell_2_Tp']
bmus = xds_WT_KMA.bmus.values

# store parameters_fit and parameters_sample (shape, location, scale)
xds_gev_params = xr.Dataset(
    coords={
        'n_cluster' : np.arange(n_clusters)+1,
        'parameter' : ['shape', 'location', 'scale'],
    }
)

# GEV KMA Frechet
for vn in vars_fit:
    
    # fit: obtain GEV parameters
    gp_pars = FitGEV(bmus, n_clusters, xds_WT_wvs_fam_noTCs[vn].values)
    
    # sample: generate sampled GEV parameters
    sm_pars = SampleGEV(bmus, n_clusters, gp_pars, xds_WT_wvs_fam_noTCs[vn].values)
    
    # store data
    xds_gev_params['{0}_fit'.format(vn)] = (('n_cluster', 'parameter',), gp_pars)
    xds_gev_params['{0}_sample'.format(vn)] = (('n_cluster', 'parameter',), sm_pars)
    
print(xds_gev_params)    


<xarray.Dataset>
Dimensions:            (n_cluster: 36, parameter: 3)
Coordinates:
  * n_cluster          (n_cluster) int64 1 2 3 4 5 6 7 ... 30 31 32 33 34 35 36
  * parameter          (parameter) <U8 'shape' 'location' 'scale'
Data variables:
    sea_Hs_fit         (n_cluster, parameter) float64 0.2375 0.1898 ... 0.1431
    sea_Hs_sample      (n_cluster, parameter) float64 0.2375 0.1898 ... 0.1431
    sea_Tp_fit         (n_cluster, parameter) float64 0.6145 2.906 ... 1.164
    sea_Tp_sample      (n_cluster, parameter) float64 0.6145 2.906 ... 1.164
    swell_1_Hs_fit     (n_cluster, parameter) float64 -0.005129 ... 0.4409
    swell_1_Hs_sample  (n_cluster, parameter) float64 -0.005129 ... 0.4409
    swell_1_Tp_fit     (n_cluster, parameter) float64 -0.4158 12.08 ... 1.395
    swell_1_Tp_sample  (n_cluster, parameter) float64 -0.4158 12.08 ... 1.395
    swell_2_Hs_fit     (n_cluster, parameter) float64 -0.03568 1.712 ... 0.4727
    swell_2_Hs_sample  (n_cluster, parameter) float64 -0.

In [5]:
# --------------------------------------
# Calculate chromosomes and chromosomes probabilities

vars_chrom = ['sea_Hs', 'swell_1_Hs', 'swell_2_Hs'] 

np_vars_chrom = np.column_stack(
    [xds_WT_wvs_fam_noTCs[vn].values for vn in vars_chrom]
)
chrom, chrom_probs = ChromProbs(bmus, n_clusters, np_vars_chrom)

print(chrom)
print()
print(chrom_probs)


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]

[[0.         0.01724138 0.01724138 0.         0.43103448 0.
  0.53448276]
 [0.         0.         0.         0.         0.55263158 0.02631579
  0.42105263]
 [0.         0.         0.00952381 0.         0.43809524 0.03809524
  0.4952381 ]
 [0.         0.         0.03278689 0.         0.3852459  0.00819672
  0.56557377]
 [0.         0.01941748 0.02912621 0.         0.40776699 0.03883495
  0.44660194]
 [0.         0.         0.03921569 0.         0.47058824 0.09803922
  0.31372549]
 [0.         0.         0.00769231 0.         0.54615385 0.
  0.43846154]
 [0.         0.         0.01449275 0.         0.39855072 0.00724638
  0.56521739]
 [0.         0.         0.00595238 0.         0.41666667 0.01785714
  0.55952381]
 [0.         0.         0.03606557 0.         0.2295082  0.12131148
  0.60655738]
 [0.         0.00269542 0.04312668 0.         0.40700809 0.06199461
  0.43665768]
 [0.         0.         0.  

In [6]:
# --------------------------------------
# Simulation

# TODO: persistences for statisticals: mean duration, prctile....
#d_pers = Persistences(bmus)

